<a href="https://colab.research.google.com/github/WooHyunJ/Final-Team9/blob/main/9%EC%9B%9424%EC%9D%BC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [42]:
import pandas as pd

In [43]:
df = pd.read_csv("/content/drive/MyDrive/TMDB_processed_final.csv")

In [44]:
df.columns

Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'budget', 'imdb_id', 'original_language',
       'original_title', 'overview', 'popularity', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords', 'director', 'cast', 'ROI', 'SR', 'y_result'],
      dtype='object')

In [45]:
df['release_date'] = pd.to_datetime(df['release_date'], errors='coerce')
# 2. 날짜 형식으로 변환된 컬럼에서 '월'과 '분기' 정보 추출
# .dt 접근자를 사용하면 날짜 관련 정보를 쉽게 뽑아낼 수 있습니다.
df['month'] = df['release_date'].dt.month
df['quarter'] = df['release_date'].dt.quarter

# 'month', 'quarter' 컬럼 생성
print(df[['release_date', 'month', 'quarter']].head())

  release_date  month  quarter
0   2010-07-15      7        3
1   2014-11-05     11        4
2   2008-07-16      7        3
3   2009-12-15     12        4
4   2012-04-25      4        2


In [46]:
import pandas as pd
from scipy.stats import chi2_contingency

# --- '월(month)'과 'Y Result' 간의 카이제곱 검정 ---

print("--- '월(month)' 기준 카이제곱 검정 ---")

# 1. 교차표(Contingency Table) 생성
# 행(index)에는 흥행 등급, 열(column)에는 개봉 월을 배치합니다.
contingency_table_month = pd.crosstab(df['y_result'], df['month'])

print("\n[월별 흥행 등급 교차표]")
print(contingency_table_month)

# 2. 카이제곱 검정 실행
# 교차표를 chi2_contingency 함수에 전달합니다.
chi2, p_value, dof, expected = chi2_contingency(contingency_table_month)

print(f"\n카이제곱 통계량(Chi-squared Statistic): {chi2:.4f}")
print(f"p-value: {p_value:.4f}")
print(f"자유도(Degrees of Freedom): {dof}")

# 3. 결과 해석
alpha = 0.05  # 유의수준 5%
if p_value < alpha:
    print("\n[결론] p-value가 유의수준보다 작으므로, '개봉 월'과 '흥행'은 통계적으로 유의미한 관련이 있습니다.")
else:
    print("\n[결론] p-value가 유의수준보다 크므로, '개봉 월'과 '흥행'의 관련성을 통계적으로 입증하지 못했습니다.")

--- '월(month)' 기준 카이제곱 검정 ---

[월별 흥행 등급 교차표]
month      1    2    3    4    5    6    7    8    9    10   11   12
y_result                                                            
0         176  200  212  214  206  179  161  254  413  332  221  245
1         141  166  196  189  183  175  197  213  235  224  182  248
2         130  152  171  146  153  196  179  195  179  162  150  212
3          44   63   72   51   91  132  114   61   72   91  100  165

카이제곱 통계량(Chi-squared Statistic): 232.6134
p-value: 0.0000
자유도(Degrees of Freedom): 33

[결론] p-value가 유의수준보다 작으므로, '개봉 월'과 '흥행'은 통계적으로 유의미한 관련이 있습니다.


In [47]:
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

# 이전에 생성한 교차표 'contingency_table_month'가 있다고 가정합니다.
# contingency_table_month = pd.crosstab(df['Y Result'], df['month'])

# 1. 카이제곱 검정 재실행하여 '기대 빈도' 값 얻기
chi2, p, dof, expected = chi2_contingency(contingency_table_month)

# 2. 표준화 잔차 계산
# 공식: (관측 빈도 - 기대 빈도) / sqrt(기대 빈도)
residuals = (contingency_table_month - expected) / np.sqrt(expected)

# 더 정확한 분석을 위해 조정 잔차(Adjusted Residuals)를 사용하는 경우도 많지만,
# 해석의 편의성을 위해 표준화 잔차로도 충분히 의미있는 결과를 얻을 수 있습니다.
# 여기서는 표준화 잔차(residuals)를 사용하겠습니다.

print("--- 표준화 잔차 (Standardized Residuals) ---")
print(residuals)


# 3. (선택사항) 결과를 보기 좋게 시각화하기 (강력 추천!)
# 잔차 값이 1.96보다 크거나 -1.96보다 작은 셀을 하이라이트하여 보여줍니다.
def highlight_significant_cells(val):
    if val > 1.96:
        return 'background-color: lightgreen' # 유의미하게 많음 (긍정적)
    elif val < -1.96:
        return 'background-color: lightcoral'  # 유의미하게 적음 (부정적)
    else:
        return ''

styled_residuals = residuals.style.applymap(highlight_significant_cells)

print("\n\n--- 유의미한 셀 하이라이트 ---")
# Jupyter Notebook이나 Colab 환경에서 실행하면 색상이 입혀진 표가 보입니다.
display(styled_residuals)

--- 표준화 잔차 (Standardized Residuals) ---
month           1         2         3         4         5         6   \
y_result                                                               
0         0.652150  0.122746 -0.681634  0.646046 -0.681551 -3.522521   
1         0.091319  0.033649  0.769812  1.377989  0.194671 -1.387923   
2         0.854020  0.775912  0.875630 -0.115132 -0.200853  2.198568   
3        -2.383215 -1.324990 -1.248182 -2.950198  1.100169  4.774678   

month           7         8         9         10        11        12  
y_result                                                              
0        -4.103299  0.462804  6.063661  3.365585 -0.123413 -3.011815  
1         0.843232  0.485401 -1.323715 -0.430728 -0.299444  0.004885  
2         1.508230  1.304524 -2.816154 -2.606220 -0.822536 -0.118117  
3         3.350882 -3.285784 -4.022630 -1.241599  1.787062  5.071933  


--- 유의미한 셀 하이라이트 ---


/tmp/ipython-input-2069184899.py:33: FutureWarning: Styler.applymap has been deprecated. Use Styler.map instead.
  styled_residuals = residuals.style.applymap(highlight_significant_cells)


month,1,2,3,4,5,6,7,8,9,10,11,12
y_result,,,,,,,,,,,,
0,0.652150,0.122746,-0.681634,0.646046,-0.681551,-3.522521,-4.103299,0.462804,6.063661,3.365585,-0.123413,-3.011815
1,0.091319,0.033649,0.769812,1.377989,0.194671,-1.387923,0.843232,0.485401,-1.323715,-0.430728,-0.299444,0.004885
2,0.854020,0.775912,0.875630,-0.115132,-0.200853,2.198568,1.508230,1.304524,-2.816154,-2.606220,-0.822536,-0.118117
3,-2.383215,-1.324990,-1.248182,-2.950198,1.100169,4.774678,3.350882,-3.285784,-4.022630,-1.241599,1.787062,5.071933


In [48]:
# 1. '성공 그룹' 데이터 필터링 (y_result가 2 또는 3)
success_df = df[df['y_result'].isin([2, 3])].copy()

# 2. 예산이 0보다 크고, 결측치가 없는 데이터만 남기기
success_df = success_df[success_df['budget'] > 0].dropna()

print("성공 그룹 데이터 준비 완료!")
print(f"분석에 사용할 성공 영화 개수: {len(success_df)}")

성공 그룹 데이터 준비 완료!
분석에 사용할 성공 영화 개수: 2796


In [49]:
from scipy.stats import ttest_ind

# 2단계에서 생성한 success_df 사용

# 1. '성수기'와 '비수기' 그룹으로 데이터 나누기
peak_season_months = [5, 6, 7, 11, 12]
peak_group = success_df[success_df['month'].isin(peak_season_months)]['budget']
off_peak_group = success_df[~success_df['month'].isin(peak_season_months)]['budget']

print(f"성수기 성공작 평균 예산: ${peak_group.mean():,.2f}")
print(f"비수기 성공작 평균 예산: ${off_peak_group.mean():,.2f}")

# 2. T-test 실행
ttest_result = ttest_ind(peak_group, off_peak_group, equal_var=False)
print(f"\nT-test p-value: {ttest_result.pvalue:.10f}")

# 3. 결과 해석
if ttest_result.pvalue < 0.05:
    print(">> 결론: 성수기와 비수기 성공작들의 평균 예산 차이는 통계적으로 유의미합니다.")
else:
    print(">> 결론: 두 그룹의 평균 예산 차이는 통계적으로 유의미하지 않습니다.")

성수기 성공작 평균 예산: $45,790,930.77
비수기 성공작 평균 예산: $29,048,210.87

T-test p-value: 0.0000000000
>> 결론: 성수기와 비수기 성공작들의 평균 예산 차이는 통계적으로 유의미합니다.
